# TFM

## Importación de librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import toml
import random
import tensorflow as tf
from sklearn.metrics import (
    accuracy_score, 
    f1_score,
    recall_score,
    precision_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

## Ejecución de código en GPU

In [2]:
import tensorflow as tf

# Comprobar la versión de TensorFlow
print("TensorFlow version:", tf.__version__)

# Listar todos los dispositivos físicos disponibles
physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):")
    for gpu in physical_devices:
        print(f'\t{gpu}')
else:
    print("🔴 No se ha detectado ninguna GPU. TensorFlow está utilizando la CPU.")

TensorFlow version: 2.10.0
🟢 TensorFlow ha detectado la(s) siguiente(s) GPU(s):
	PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Lectura de fichero

In [3]:
data_dict = {}
DATA_DIR = '../../data/'

for data_folder in os.listdir(DATA_DIR):
    *data_files, config_file = os.listdir(f'{DATA_DIR}/{data_folder}')

    leak_value = toml.load(f'{DATA_DIR}/{data_folder}/{config_file}')['tanks']['flow_value']

    train_dataframe, test_data = random.sample(data_files, 2)

    data_dict[data_folder] = {
        'leak_value': leak_value,
        'train_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{train_dataframe}').drop(columns=['Unnamed: 0']),
        'test_dataframe': pd.read_csv(f'{DATA_DIR}/{data_folder}/{test_data}').drop(columns=['Unnamed: 0'])
    }

### Conjuntos de datos de entrenamiento

In [4]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21950,34",-6035,"7006,85","22922,19","22962,99","40,8","172,2",0
1,2,"22962,99",-6105,"6515,76","23373,75","23300,26","-73,49","125,79",0
2,3,"23300,26",-5770,"6438,69","23968,95","24020,7","51,75","97,38",0
3,4,"24020,7",-5915,"5427,88","23533,58","23536,74","3,16","79,51",0
4,5,"23536,74",-6415,"4351,1","21472,84","21473,49","0,65","87,66",0
...,...,...,...,...,...,...,...,...,...
995,996,"15927,66",-6095,"6450,17","16282,83","16239,24","-43,59","114,14",1
996,997,"16239,24",-5855,"6019,11","16403,35","16385,84","-17,51","38,4",1
997,998,"16385,84",-6120,"3750,83","14016,67","13979,49","-37,18","-21,93",1
998,999,"13979,49",-5845,"6725,82","14860,31","14857,03","-3,28","10,77",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22430,06","-6075,0","5976,32","22331,38","22376,0","44,62","-116,83",0
1,2,"22376,0","-6065,0","3965,07","20276,07","20290,81","14,74","-45,27",0
2,3,"20290,81","-5775,0","6982,55","21498,36","21513,87","15,51","-60,91",0
3,4,"21513,87","-6035,0","6956,14","22435,01","22426,74","-8,27","5,59",0
4,5,"22426,74","-6030,0","6377,94","22774,68","22817,15","42,47","36,97",0
...,...,...,...,...,...,...,...,...,...
995,996,"2107,2","-5828,98","6463,64","2741,86","2620,44","-121,42","-167,54",1
996,997,"2620,44","-5955,0","6142,6","2808,04","2771,02","-37,02","-143,07",1
997,998,"2771,02","-5896,0","3108,82","-16,16","25,62","41,78","-150,11",1
998,999,"25,62","-5592,26","6421,37","854,73","896,61","41,88","-83,97",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"10484,66","-5975,0","7217,79","11727,45","11882,71","155,26","381,21",0
1,2,"11882,71","-5850,0","7035,09","13067,8","13045,34","-22,46","344,85",0
2,3,"13045,34","-6140,0","3567,08","10472,42","10500,66","28,24","325,67",0
3,4,"10500,66","-6085,0","7441,43","11857,09","11819,52","-37,57","344,23",0
4,5,"11819,52","-6005,0","8124,72","13939,24","13815,32","-123,92","200,7",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,4","-0,0","0,0","3,4","2,76","-0,64","-20,06",1
996,997,"2,76","-0,0","0,0","2,76","9,57","6,81","-7,51",1
997,998,"9,57","-0,0","0,0","9,57","11,33","1,76","6,47",1
998,999,"11,33","-0,0","0,0","11,33","17,02","5,69","9,31",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21399,05","-5790,0","5638,02","21247,07","21227,09","-19,98","63,76",0
1,2,"21227,09","-5895,0","5241,74","20573,83","20576,89","3,06","133,61",0
2,3,"20576,89","-6055,0","6999,43","21521,32","21504,09","-17,23","33,97",0
3,4,"21504,09","-5995,0","9004,73","24513,82","24667,5","153,68","232,55",0
4,5,"24667,5","-6005,0","5447,75","24110,25","24146,71","36,46","202,77",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,62","-0,0","0,0","2,62","19,24","16,62","5,27",1
996,997,"19,24","-0,0","0,0","19,24","14,88","-4,36","5,99",1
997,998,"14,88","-0,0","0,0","14,88","9,37","-5,51","-4,99",1
998,999,"9,37","-0,0","0,0","9,37","3,89","-5,48","-11,62",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"21372,17","-5985,0","7788,9","23176,07","23214,64","38,57","19,95",0
1,2,"23214,64","-5765,0","2969,27","20418,91","20415,11","-3,8","61,14",0
2,3,"20415,11","-6325,0","7390,24","21480,35","21602,29","121,94","144,09",0
3,4,"21602,29","-6190,0","7315,88","22728,17","22691,65","-36,52","217,46",0
4,5,"22691,65","-5690,0","6031,24","23032,89","22983,03","-49,86","170,15",0
...,...,...,...,...,...,...,...,...,...
995,996,"17,19","-0,0","0,0","17,19","26,77","9,58","23,15",1
996,997,"26,77","-0,0","0,0","26,77","11,55","-15,22","10,12",1
997,998,"11,55","-0,0","0,0","11,55","1,45","-10,1","-1,74",1
998,999,"1,45","-0,0","0,0","1,45","8,2","6,75","5,73",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17410,82","-5735,0","7973,05","19648,87","19662,73","13,86","280,74",0
1,2,"19662,73","-6395,0","6009,46","19277,19","19276,39","-0,8","204,46",0
2,3,"19276,39","-5665,0","8990,36","22601,75","22599,67","-2,08","37,4",0
3,4,"22599,67","-6365,0","6085,67","22320,34","22238,26","-82,08","0,27",0
4,5,"22238,26","-6175,0","6515,23","22578,49","22661,17","82,68","148,25",0
...,...,...,...,...,...,...,...,...,...
995,996,"20,64","-0,0","0,0","20,64","3,15","-17,49","-13,37",1
996,997,"3,15","-0,0","0,0","3,15","11,7","8,55","0,13",1
997,998,"11,7","-0,0","0,0","11,7","8,79","-2,91","4,5",1
998,999,"8,79","-0,0","0,0","8,79","13,8","5,01","12,43",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"18109,75","-5940,0","6999,66","19169,41","19140,47","-28,94","290,5",0
1,2,"19140,47","-6320,0","7014,09","19834,56","19825,75","-8,81","346,46",0
2,3,"19825,75","-6145,0","7026,85","20707,6","20677,99","-29,61","326,59",0
3,4,"20677,99","-5805,0","4783,79","19656,78","19702,96","46,18","340,89",0
4,5,"19702,96","-6195,0","6592,28","20100,24","20021,69","-78,55","117,34",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,39","-0,0","0,0","3,39","8,94","5,55","-9,07",1
996,997,"8,94","-0,0","0,0","8,94","13,46","4,52","1,87",1
997,998,"13,46","-0,0","0,0","13,46","5,2","-8,26","4,92",1
998,999,"5,2","-0,0","0,0","5,2","13,18","7,98","-1,52",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"16777,0","-6100,0","9895,09","20572,09","20589,64","17,55","-18,03",0
1,2,"20589,64","-6115,0","4955,9","19430,54","19468,61","38,07","-12,02",0
2,3,"19468,61","-5965,0","6946,41","20450,02","20512,05","62,03","14,91",0
3,4,"20512,05","-6245,0","2980,24","17247,29","17265,48","18,19","87,81",0
4,5,"17265,48","-6425,0","8878,71","19719,19","19875,47","156,28","275,63",0
...,...,...,...,...,...,...,...,...,...
995,996,"0,33","-0,0","0,0","0,33","0,14","-0,19","-6,95",1
996,997,"0,14","-0,0","0,0","0,14","10,7","10,56","-1,35",1
997,998,"10,7","-0,0","0,0","10,7","11,2","0,5","4,17",1
998,999,"11,2","-0,0","0,0","11,2","11,98","0,78","-1,3",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22682,46","-5995,0","5831,75","22519,21","22535,33","16,12","459,1",0
1,2,"22535,33","-5975,0","3507,95","20068,28","20135,89","67,61","514,24",0
2,3,"20135,89","-6250,0","8020,71","21906,6","21854,08","-52,52","421,71",0
3,4,"21854,08","-5985,0","8129,34","23998,42","23900,39","-98,03","282,93",0
4,5,"23900,39","-6030,0","5690,89","23561,28","23581,5","20,22","287,66",0
...,...,...,...,...,...,...,...,...,...
995,996,"6,96","-0,0","0,0","6,96","3,42","-3,54","-0,25",1
996,997,"3,42","-0,0","0,0","3,42","18,91","15,49","16,44",1
997,998,"18,91","-0,0","0,0","18,91","16,72","-2,19","9,09",1
998,999,"16,72","-0,0","0,0","16,72","1,44","-15,28","-0,25",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22883,35","-6050,0","5453,86","22287,21","22230,52","-56,69","249,29",0
1,2,"22230,52","-6275,0","3883,29","19838,81","19981,1","142,29","345,12",0
2,3,"19981,1","-6150,0","7525,49","21356,59","21346,06","-10,53","273,17",0
3,4,"21346,06","-6130,0","6948,29","22164,35","22227,3","62,95","280,51",0
4,5,"22227,3","-5815,0","5639,5","22051,8","22112,87","61,07","286,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,47","-0,0","0,0","1,47","15,25","13,78","10,81",1
996,997,"15,25","-0,0","0,0","15,25","6,77","-8,48","-2,84",1
997,998,"6,77","-0,0","0,0","6,77","1,04","-5,73","-11,61",1
998,999,"1,04","-0,0","0,0","1,04","22,11","21,07","14,93",1


### Conjunto de datos de prueba

In [5]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20247,25",-5580,"6051,35","20718,6","20679,11","-39,49","-2,21",0
1,2,"20679,11",-6195,"7936,54","22420,65","22497,6","76,95","164,76",0
2,3,"22497,6",-6165,"5995,68","22328,28","22284,53","-43,75","73,79",0
3,4,"22284,53",-5720,"5031,76","21596,29","21654,39","58,1","181,92",0
4,5,"21654,39",-6075,"9124,56","24703,95","24903,47","199,52","323,61",0
...,...,...,...,...,...,...,...,...,...
995,996,"14567,32",-5990,"6995,54","15572,86","15611,86","39,0","-8,11",1
996,997,"15611,86",-6405,"6226,8","15433,66","15483,99","50,33","14,33",1
997,998,"15483,99",-5670,"5799,07","15613,06","15537,81","-75,25","6,19",1
998,999,"15537,81",-6130,"5099,01","14506,82","14515,34","8,52","84,27",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"14034,89","-5705,0","7013,6","15343,49","15323,45","-20,04","228,67",0
1,2,"15323,45","-5970,0","7633,39","16986,84","16980,09","-6,75","138,05",0
2,3,"16980,09","-6250,0","4594,55","15324,64","15342,95","18,31","91,74",0
3,4,"15342,95","-6375,0","7716,89","16684,84","16706,73","21,89","36,45",0
4,5,"16706,73","-6020,0","7060,35","17747,08","17712,72","-34,36","101,33",0
...,...,...,...,...,...,...,...,...,...
995,996,"2480,65","-5740,0","5618,89","2359,54","2424,72","65,18","115,61",1
996,997,"2424,72","-5725,0","6292,34","2992,06","2952,52","-39,54","66,08",1
997,998,"2952,52","-5377,62","2403,34","-21,76","8,21","29,97","58,91",1
998,999,"8,21","-5756,65","6559,47","811,03","872,06","61,03","81,69",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,22","-5810,0","7001,2","23334,42","23347,02","12,6","-90,27",0
1,2,"23347,02","-6240,0","7270,93","24377,95","24283,54","-94,41","-105,27",0
2,3,"24283,54","-6430,0","3650,03","21503,57","21510,13","6,56","-17,7",0
3,4,"21510,13","-5850,0","6940,97","22601,1","22644,04","42,94","27,91",0
4,5,"22644,04","-6025,0","5910,84","22529,88","22565,92","36,04","80,35",0
...,...,...,...,...,...,...,...,...,...
995,996,"20,99","-0,0","0,0","20,99","7,38","-13,61","5,7",1
996,997,"7,38","-0,0","0,0","7,38","2,99","-4,39","-5,43",1
997,998,"2,99","-0,0","0,0","2,99","14,37","11,38","9,91",1
998,999,"14,37","-0,0","0,0","14,37","23,46","9,09","10,89",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"13689,29","-6200,0","9859,61","17348,9","17315,01","-33,89","93,19",0
1,2,"17315,01","-5965,0","6578,38","17928,39","18121,96","193,57","276,59",0
2,3,"18121,96","-6170,0","5339,44","17291,4","17357,0","65,6","264,32",0
3,4,"17357,0","-5975,0","6966,25","18348,25","18365,14","16,89","200,58",0
4,5,"18365,14","-6135,0","7964,66","20194,8","20222,38","27,58","192,82",0
...,...,...,...,...,...,...,...,...,...
995,996,"14,6","-0,0","0,0","14,6","3,95","-10,65","-1,28",1
996,997,"3,95","-0,0","0,0","3,95","8,56","4,61","-6,84",1
997,998,"8,56","-0,0","0,0","8,56","1,51","-7,05","-2,43",1
998,999,"1,51","-0,0","0,0","1,51","15,58","14,07","12,31",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"20535,61","-5940,0","8641,12","23236,73","23294,75","58,02","265,22",0
1,2,"23294,75","-6575,0","6455,26","23175,01","23296,05","121,04","387,15",0
2,3,"23296,05","-5630,0","6600,92","24266,97","24325,7","58,73","366,14",0
3,4,"24325,7","-6015,0","3190,31","21501,01","21523,46","22,45","375,09",0
4,5,"21523,46","-5695,0","7376,83","23205,29","23238,07","32,78","376,68",0
...,...,...,...,...,...,...,...,...,...
995,996,"3,69","-0,0","0,0","3,69","23,8","20,11","18,39",1
996,997,"23,8","-0,0","0,0","23,8","15,45","-8,35","12,43",1
997,998,"15,45","-0,0","0,0","15,45","10,85","-4,6","-0,6",1
998,999,"10,85","-0,0","0,0","10,85","3,49","-7,36","-0,14",1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"22143,65","-6055,0","6215,36","22304,01","22242,32","-61,69","34,11",0
1,2,"22242,32","-5940,0","6691,65","22993,97","22983,66","-10,31","42,41",0
2,3,"22983,66","-5885,0","6130,98","23229,64","23301,88","72,24","58,48",0
3,4,"23301,88","-6030,0","5035,77","22307,65","22270,88","-36,77","25,47",0
4,5,"22270,88","-6005,0","6518,15","22784,03","22794,42","10,39","13,96",0
...,...,...,...,...,...,...,...,...,...
995,996,"2,74","-0,0","0,0","2,74","3,88","1,14","3,0",1
996,997,"3,88","-0,0","0,0","3,88","0,72","-3,16","-2,72",1
997,998,"0,72","-0,0","0,0","0,72","1,13","0,41","-11,02",1
998,999,"1,13","-0,0","0,0","1,13","2,14","1,01","-19,34",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15886,17","-5900,0","5970,3","15956,47","15980,01","23,54","68,46",0
1,2,"15980,01","-5810,0","6050,74","16220,75","16175,65","-45,1","8,64",0
2,3,"16175,65","-5760,0","5917,51","16333,16","16413,96","80,8","17,3",0
3,4,"16413,96","-5970,0","6982,8","17426,76","17460,02","33,26","68,23",0
4,5,"17460,02","-6340,0","7929,64","19049,66","19111,41","61,75","52,84",0
...,...,...,...,...,...,...,...,...,...
995,996,"5,52","-0,0","0,0","5,52","0,47","-5,05","-4,84",1
996,997,"0,47","-0,0","0,0","0,47","5,79","5,32","0,68",1
997,998,"5,79","-0,0","0,0","5,79","14,14","8,35","-0,25",1
998,999,"14,14","-0,0","0,0","14,14","4,73","-9,41","2,66",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"15272,5","-6010,0","10030,29","19292,79","19280,8","-11,99","292,48",0
1,2,"19280,8","-6205,0","3984,78","17060,58","17094,5","33,92","305,91",0
2,3,"17094,5","-5995,0","7951,55","19051,05","19091,82","40,77","164,52",0
3,4,"19091,82","-6255,0","3983,16","16819,98","16872,08","52,1","215,19",0
4,5,"16872,08","-5965,0","8512,19","19419,27","19433,93","14,66","140,65",0
...,...,...,...,...,...,...,...,...,...
995,996,"16,81","-0,0","0,0","16,81","4,18","-12,63","-0,89",1
996,997,"4,18","-0,0","0,0","4,18","10,01","5,83","5,26",1
997,998,"10,01","-0,0","0,0","10,01","6,14","-3,87","-0,29",1
998,999,"6,14","-0,0","0,0","6,14","14,1","7,96","2,67",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"17698,38","-5710,0","4618,64","16607,02","16580,51","-26,51","35,66",0
1,2,"16580,51","-5875,0","6004,3","16709,81","16702,31","-7,5","58,61",0
2,3,"16702,31","-6250,0","8001,28","18453,59","18456,14","2,55","-9,07",0
3,4,"18456,14","-5870,0","6931,83","19517,97","19595,08","77,11","-16,24",0
4,5,"19595,08","-6330,0","7449,42","20714,5","20644,03","-70,47","-108,89",0
...,...,...,...,...,...,...,...,...,...
995,996,"1,11","-0,0","0,0","1,11","1,53","0,42","-12,95",1
996,997,"1,53","-0,0","0,0","1,53","5,18","3,65","0,97",1
997,998,"5,18","-0,0","0,0","5,18","9,61","4,43","9,4",1
998,999,"9,61","-0,0","0,0","9,61","13,39","3,78","3,06",1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,"11373,11","-5975,0","6995,25","12393,36","12436,56","43,2","4,09",0
1,2,"12436,56","-6120,0","7067,55","13384,11","13284,95","-99,16","-89,11",0
2,3,"13284,95","-5925,0","7087,68","14447,63","14371,67","-75,96","-172,46",0
3,4,"14371,67","-5840,0","11960,98","20492,65","20550,77","58,12","-132,89",0
4,5,"20550,77","-6070,0","7059,92","21540,69","21530,27","-10,42","-122,12",0
...,...,...,...,...,...,...,...,...,...
995,996,"11,82","-0,0","0,0","11,82","11,43","-0,39","-5,83",1
996,997,"11,43","-0,0","0,0","11,43","13,08","1,65","10,37",1
997,998,"13,08","-0,0","0,0","13,08","1,18","-11,9","-5,79",1
998,999,"1,18","-0,0","0,0","1,18","20,33","19,15","11,23",1


## Limpieza de datos

In [6]:
bad_formatted_fields = [
    'Volumen dep. almacenam. ini. (L)',
    'Llenado dep. almacenam. (L)',
    'Volumen dep. almacenam. fin. teor. (L)',
    'Volumen dep. almacenam. fin. (L)',
    'Variacion',
    'Variacion Acum.',
]

for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        for field in bad_formatted_fields:
            data_dict[case][dataset][field] = data_dict[case][dataset][field].str.replace(',', '.')
            data_dict[case][dataset][field] = pd.to_numeric(data_dict[case][dataset][field])

### Conjunto de datos de entrenamiento

In [7]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21950.34,-6035,7006.85,22922.19,22962.99,40.80,172.20,0
1,2,22962.99,-6105,6515.76,23373.75,23300.26,-73.49,125.79,0
2,3,23300.26,-5770,6438.69,23968.95,24020.70,51.75,97.38,0
3,4,24020.70,-5915,5427.88,23533.58,23536.74,3.16,79.51,0
4,5,23536.74,-6415,4351.10,21472.84,21473.49,0.65,87.66,0
...,...,...,...,...,...,...,...,...,...
995,996,15927.66,-6095,6450.17,16282.83,16239.24,-43.59,114.14,1
996,997,16239.24,-5855,6019.11,16403.35,16385.84,-17.51,38.40,1
997,998,16385.84,-6120,3750.83,14016.67,13979.49,-37.18,-21.93,1
998,999,13979.49,-5845,6725.82,14860.31,14857.03,-3.28,10.77,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22430.06,"-6075,0",5976.32,22331.38,22376.00,44.62,-116.83,0
1,2,22376.00,"-6065,0",3965.07,20276.07,20290.81,14.74,-45.27,0
2,3,20290.81,"-5775,0",6982.55,21498.36,21513.87,15.51,-60.91,0
3,4,21513.87,"-6035,0",6956.14,22435.01,22426.74,-8.27,5.59,0
4,5,22426.74,"-6030,0",6377.94,22774.68,22817.15,42.47,36.97,0
...,...,...,...,...,...,...,...,...,...
995,996,2107.20,"-5828,98",6463.64,2741.86,2620.44,-121.42,-167.54,1
996,997,2620.44,"-5955,0",6142.60,2808.04,2771.02,-37.02,-143.07,1
997,998,2771.02,"-5896,0",3108.82,-16.16,25.62,41.78,-150.11,1
998,999,25.62,"-5592,26",6421.37,854.73,896.61,41.88,-83.97,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,10484.66,"-5975,0",7217.79,11727.45,11882.71,155.26,381.21,0
1,2,11882.71,"-5850,0",7035.09,13067.80,13045.34,-22.46,344.85,0
2,3,13045.34,"-6140,0",3567.08,10472.42,10500.66,28.24,325.67,0
3,4,10500.66,"-6085,0",7441.43,11857.09,11819.52,-37.57,344.23,0
4,5,11819.52,"-6005,0",8124.72,13939.24,13815.32,-123.92,200.70,0
...,...,...,...,...,...,...,...,...,...
995,996,3.40,"-0,0",0.00,3.40,2.76,-0.64,-20.06,1
996,997,2.76,"-0,0",0.00,2.76,9.57,6.81,-7.51,1
997,998,9.57,"-0,0",0.00,9.57,11.33,1.76,6.47,1
998,999,11.33,"-0,0",0.00,11.33,17.02,5.69,9.31,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21399.05,"-5790,0",5638.02,21247.07,21227.09,-19.98,63.76,0
1,2,21227.09,"-5895,0",5241.74,20573.83,20576.89,3.06,133.61,0
2,3,20576.89,"-6055,0",6999.43,21521.32,21504.09,-17.23,33.97,0
3,4,21504.09,"-5995,0",9004.73,24513.82,24667.50,153.68,232.55,0
4,5,24667.50,"-6005,0",5447.75,24110.25,24146.71,36.46,202.77,0
...,...,...,...,...,...,...,...,...,...
995,996,2.62,"-0,0",0.00,2.62,19.24,16.62,5.27,1
996,997,19.24,"-0,0",0.00,19.24,14.88,-4.36,5.99,1
997,998,14.88,"-0,0",0.00,14.88,9.37,-5.51,-4.99,1
998,999,9.37,"-0,0",0.00,9.37,3.89,-5.48,-11.62,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,21372.17,"-5985,0",7788.90,23176.07,23214.64,38.57,19.95,0
1,2,23214.64,"-5765,0",2969.27,20418.91,20415.11,-3.80,61.14,0
2,3,20415.11,"-6325,0",7390.24,21480.35,21602.29,121.94,144.09,0
3,4,21602.29,"-6190,0",7315.88,22728.17,22691.65,-36.52,217.46,0
4,5,22691.65,"-5690,0",6031.24,23032.89,22983.03,-49.86,170.15,0
...,...,...,...,...,...,...,...,...,...
995,996,17.19,"-0,0",0.00,17.19,26.77,9.58,23.15,1
996,997,26.77,"-0,0",0.00,26.77,11.55,-15.22,10.12,1
997,998,11.55,"-0,0",0.00,11.55,1.45,-10.10,-1.74,1
998,999,1.45,"-0,0",0.00,1.45,8.20,6.75,5.73,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17410.82,"-5735,0",7973.05,19648.87,19662.73,13.86,280.74,0
1,2,19662.73,"-6395,0",6009.46,19277.19,19276.39,-0.80,204.46,0
2,3,19276.39,"-5665,0",8990.36,22601.75,22599.67,-2.08,37.40,0
3,4,22599.67,"-6365,0",6085.67,22320.34,22238.26,-82.08,0.27,0
4,5,22238.26,"-6175,0",6515.23,22578.49,22661.17,82.68,148.25,0
...,...,...,...,...,...,...,...,...,...
995,996,20.64,"-0,0",0.00,20.64,3.15,-17.49,-13.37,1
996,997,3.15,"-0,0",0.00,3.15,11.70,8.55,0.13,1
997,998,11.70,"-0,0",0.00,11.70,8.79,-2.91,4.50,1
998,999,8.79,"-0,0",0.00,8.79,13.80,5.01,12.43,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,18109.75,"-5940,0",6999.66,19169.41,19140.47,-28.94,290.50,0
1,2,19140.47,"-6320,0",7014.09,19834.56,19825.75,-8.81,346.46,0
2,3,19825.75,"-6145,0",7026.85,20707.60,20677.99,-29.61,326.59,0
3,4,20677.99,"-5805,0",4783.79,19656.78,19702.96,46.18,340.89,0
4,5,19702.96,"-6195,0",6592.28,20100.24,20021.69,-78.55,117.34,0
...,...,...,...,...,...,...,...,...,...
995,996,3.39,"-0,0",0.00,3.39,8.94,5.55,-9.07,1
996,997,8.94,"-0,0",0.00,8.94,13.46,4.52,1.87,1
997,998,13.46,"-0,0",0.00,13.46,5.20,-8.26,4.92,1
998,999,5.20,"-0,0",0.00,5.20,13.18,7.98,-1.52,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,16777.00,"-6100,0",9895.09,20572.09,20589.64,17.55,-18.03,0
1,2,20589.64,"-6115,0",4955.90,19430.54,19468.61,38.07,-12.02,0
2,3,19468.61,"-5965,0",6946.41,20450.02,20512.05,62.03,14.91,0
3,4,20512.05,"-6245,0",2980.24,17247.29,17265.48,18.19,87.81,0
4,5,17265.48,"-6425,0",8878.71,19719.19,19875.47,156.28,275.63,0
...,...,...,...,...,...,...,...,...,...
995,996,0.33,"-0,0",0.00,0.33,0.14,-0.19,-6.95,1
996,997,0.14,"-0,0",0.00,0.14,10.70,10.56,-1.35,1
997,998,10.70,"-0,0",0.00,10.70,11.20,0.50,4.17,1
998,999,11.20,"-0,0",0.00,11.20,11.98,0.78,-1.30,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22682.46,"-5995,0",5831.75,22519.21,22535.33,16.12,459.10,0
1,2,22535.33,"-5975,0",3507.95,20068.28,20135.89,67.61,514.24,0
2,3,20135.89,"-6250,0",8020.71,21906.60,21854.08,-52.52,421.71,0
3,4,21854.08,"-5985,0",8129.34,23998.42,23900.39,-98.03,282.93,0
4,5,23900.39,"-6030,0",5690.89,23561.28,23581.50,20.22,287.66,0
...,...,...,...,...,...,...,...,...,...
995,996,6.96,"-0,0",0.00,6.96,3.42,-3.54,-0.25,1
996,997,3.42,"-0,0",0.00,3.42,18.91,15.49,16.44,1
997,998,18.91,"-0,0",0.00,18.91,16.72,-2.19,9.09,1
998,999,16.72,"-0,0",0.00,16.72,1.44,-15.28,-0.25,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22883.35,"-6050,0",5453.86,22287.21,22230.52,-56.69,249.29,0
1,2,22230.52,"-6275,0",3883.29,19838.81,19981.10,142.29,345.12,0
2,3,19981.10,"-6150,0",7525.49,21356.59,21346.06,-10.53,273.17,0
3,4,21346.06,"-6130,0",6948.29,22164.35,22227.30,62.95,280.51,0
4,5,22227.30,"-5815,0",5639.50,22051.80,22112.87,61.07,286.84,0
...,...,...,...,...,...,...,...,...,...
995,996,1.47,"-0,0",0.00,1.47,15.25,13.78,10.81,1
996,997,15.25,"-0,0",0.00,15.25,6.77,-8.48,-2.84,1
997,998,6.77,"-0,0",0.00,6.77,1.04,-5.73,-11.61,1
998,999,1.04,"-0,0",0.00,1.04,22.11,21.07,14.93,1


### Conjunto de datos de prueba

In [8]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20247.25,-5580,6051.35,20718.60,20679.11,-39.49,-2.21,0
1,2,20679.11,-6195,7936.54,22420.65,22497.60,76.95,164.76,0
2,3,22497.60,-6165,5995.68,22328.28,22284.53,-43.75,73.79,0
3,4,22284.53,-5720,5031.76,21596.29,21654.39,58.10,181.92,0
4,5,21654.39,-6075,9124.56,24703.95,24903.47,199.52,323.61,0
...,...,...,...,...,...,...,...,...,...
995,996,14567.32,-5990,6995.54,15572.86,15611.86,39.00,-8.11,1
996,997,15611.86,-6405,6226.80,15433.66,15483.99,50.33,14.33,1
997,998,15483.99,-5670,5799.07,15613.06,15537.81,-75.25,6.19,1
998,999,15537.81,-6130,5099.01,14506.82,14515.34,8.52,84.27,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,14034.89,"-5705,0",7013.60,15343.49,15323.45,-20.04,228.67,0
1,2,15323.45,"-5970,0",7633.39,16986.84,16980.09,-6.75,138.05,0
2,3,16980.09,"-6250,0",4594.55,15324.64,15342.95,18.31,91.74,0
3,4,15342.95,"-6375,0",7716.89,16684.84,16706.73,21.89,36.45,0
4,5,16706.73,"-6020,0",7060.35,17747.08,17712.72,-34.36,101.33,0
...,...,...,...,...,...,...,...,...,...
995,996,2480.65,"-5740,0",5618.89,2359.54,2424.72,65.18,115.61,1
996,997,2424.72,"-5725,0",6292.34,2992.06,2952.52,-39.54,66.08,1
997,998,2952.52,"-5377,62",2403.34,-21.76,8.21,29.97,58.91,1
998,999,8.21,"-5756,65",6559.47,811.03,872.06,61.03,81.69,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.22,"-5810,0",7001.20,23334.42,23347.02,12.60,-90.27,0
1,2,23347.02,"-6240,0",7270.93,24377.95,24283.54,-94.41,-105.27,0
2,3,24283.54,"-6430,0",3650.03,21503.57,21510.13,6.56,-17.70,0
3,4,21510.13,"-5850,0",6940.97,22601.10,22644.04,42.94,27.91,0
4,5,22644.04,"-6025,0",5910.84,22529.88,22565.92,36.04,80.35,0
...,...,...,...,...,...,...,...,...,...
995,996,20.99,"-0,0",0.00,20.99,7.38,-13.61,5.70,1
996,997,7.38,"-0,0",0.00,7.38,2.99,-4.39,-5.43,1
997,998,2.99,"-0,0",0.00,2.99,14.37,11.38,9.91,1
998,999,14.37,"-0,0",0.00,14.37,23.46,9.09,10.89,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,13689.29,"-6200,0",9859.61,17348.90,17315.01,-33.89,93.19,0
1,2,17315.01,"-5965,0",6578.38,17928.39,18121.96,193.57,276.59,0
2,3,18121.96,"-6170,0",5339.44,17291.40,17357.00,65.60,264.32,0
3,4,17357.00,"-5975,0",6966.25,18348.25,18365.14,16.89,200.58,0
4,5,18365.14,"-6135,0",7964.66,20194.80,20222.38,27.58,192.82,0
...,...,...,...,...,...,...,...,...,...
995,996,14.60,"-0,0",0.00,14.60,3.95,-10.65,-1.28,1
996,997,3.95,"-0,0",0.00,3.95,8.56,4.61,-6.84,1
997,998,8.56,"-0,0",0.00,8.56,1.51,-7.05,-2.43,1
998,999,1.51,"-0,0",0.00,1.51,15.58,14.07,12.31,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,20535.61,"-5940,0",8641.12,23236.73,23294.75,58.02,265.22,0
1,2,23294.75,"-6575,0",6455.26,23175.01,23296.05,121.04,387.15,0
2,3,23296.05,"-5630,0",6600.92,24266.97,24325.70,58.73,366.14,0
3,4,24325.70,"-6015,0",3190.31,21501.01,21523.46,22.45,375.09,0
4,5,21523.46,"-5695,0",7376.83,23205.29,23238.07,32.78,376.68,0
...,...,...,...,...,...,...,...,...,...
995,996,3.69,"-0,0",0.00,3.69,23.80,20.11,18.39,1
996,997,23.80,"-0,0",0.00,23.80,15.45,-8.35,12.43,1
997,998,15.45,"-0,0",0.00,15.45,10.85,-4.60,-0.60,1
998,999,10.85,"-0,0",0.00,10.85,3.49,-7.36,-0.14,1


...

,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,22143.65,"-6055,0",6215.36,22304.01,22242.32,-61.69,34.11,0
1,2,22242.32,"-5940,0",6691.65,22993.97,22983.66,-10.31,42.41,0
2,3,22983.66,"-5885,0",6130.98,23229.64,23301.88,72.24,58.48,0
3,4,23301.88,"-6030,0",5035.77,22307.65,22270.88,-36.77,25.47,0
4,5,22270.88,"-6005,0",6518.15,22784.03,22794.42,10.39,13.96,0
...,...,...,...,...,...,...,...,...,...
995,996,2.74,"-0,0",0.00,2.74,3.88,1.14,3.00,1
996,997,3.88,"-0,0",0.00,3.88,0.72,-3.16,-2.72,1
997,998,0.72,"-0,0",0.00,0.72,1.13,0.41,-11.02,1
998,999,1.13,"-0,0",0.00,1.13,2.14,1.01,-19.34,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15886.17,"-5900,0",5970.30,15956.47,15980.01,23.54,68.46,0
1,2,15980.01,"-5810,0",6050.74,16220.75,16175.65,-45.10,8.64,0
2,3,16175.65,"-5760,0",5917.51,16333.16,16413.96,80.80,17.30,0
3,4,16413.96,"-5970,0",6982.80,17426.76,17460.02,33.26,68.23,0
4,5,17460.02,"-6340,0",7929.64,19049.66,19111.41,61.75,52.84,0
...,...,...,...,...,...,...,...,...,...
995,996,5.52,"-0,0",0.00,5.52,0.47,-5.05,-4.84,1
996,997,0.47,"-0,0",0.00,0.47,5.79,5.32,0.68,1
997,998,5.79,"-0,0",0.00,5.79,14.14,8.35,-0.25,1
998,999,14.14,"-0,0",0.00,14.14,4.73,-9.41,2.66,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,15272.50,"-6010,0",10030.29,19292.79,19280.80,-11.99,292.48,0
1,2,19280.80,"-6205,0",3984.78,17060.58,17094.50,33.92,305.91,0
2,3,17094.50,"-5995,0",7951.55,19051.05,19091.82,40.77,164.52,0
3,4,19091.82,"-6255,0",3983.16,16819.98,16872.08,52.10,215.19,0
4,5,16872.08,"-5965,0",8512.19,19419.27,19433.93,14.66,140.65,0
...,...,...,...,...,...,...,...,...,...
995,996,16.81,"-0,0",0.00,16.81,4.18,-12.63,-0.89,1
996,997,4.18,"-0,0",0.00,4.18,10.01,5.83,5.26,1
997,998,10.01,"-0,0",0.00,10.01,6.14,-3.87,-0.29,1
998,999,6.14,"-0,0",0.00,6.14,14.10,7.96,2.67,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,17698.38,"-5710,0",4618.64,16607.02,16580.51,-26.51,35.66,0
1,2,16580.51,"-5875,0",6004.30,16709.81,16702.31,-7.50,58.61,0
2,3,16702.31,"-6250,0",8001.28,18453.59,18456.14,2.55,-9.07,0
3,4,18456.14,"-5870,0",6931.83,19517.97,19595.08,77.11,-16.24,0
4,5,19595.08,"-6330,0",7449.42,20714.50,20644.03,-70.47,-108.89,0
...,...,...,...,...,...,...,...,...,...
995,996,1.11,"-0,0",0.00,1.11,1.53,0.42,-12.95,1
996,997,1.53,"-0,0",0.00,1.53,5.18,3.65,0.97,1
997,998,5.18,"-0,0",0.00,5.18,9.61,4.43,9.40,1
998,999,9.61,"-0,0",0.00,9.61,13.39,3.78,3.06,1


,Tiempo (dia),Volumen dep. almacenam. ini. (L),Venta (L),Llenado dep. almacenam. (L),Volumen dep. almacenam. fin. teor. (L),Volumen dep. almacenam. fin. (L),Variacion,Variacion Acum.,Fugando combustible
0,1,11373.11,"-5975,0",6995.25,12393.36,12436.56,43.20,4.09,0
1,2,12436.56,"-6120,0",7067.55,13384.11,13284.95,-99.16,-89.11,0
2,3,13284.95,"-5925,0",7087.68,14447.63,14371.67,-75.96,-172.46,0
3,4,14371.67,"-5840,0",11960.98,20492.65,20550.77,58.12,-132.89,0
4,5,20550.77,"-6070,0",7059.92,21540.69,21530.27,-10.42,-122.12,0
...,...,...,...,...,...,...,...,...,...
995,996,11.82,"-0,0",0.00,11.82,11.43,-0.39,-5.83,1
996,997,11.43,"-0,0",0.00,11.43,13.08,1.65,10.37,1
997,998,13.08,"-0,0",0.00,13.08,1.18,-11.90,-5.79,1
998,999,1.18,"-0,0",0.00,1.18,20.33,19.15,11.23,1


<!-- TODO -->

## Selección de características

In [9]:
for case in data_dict:
    for dataset in ['train_dataframe', 'test_dataframe']:
        data_dict[case][dataset] = data_dict[case][dataset].filter(items=['Variacion', 'Fugando combustible'])

### Conjunto de datos de entrenamiento

In [10]:
for case in data_dict:
    display(data_dict[case]['train_dataframe'])

,Variacion,Fugando combustible
0,40.80,0
1,-73.49,0
2,51.75,0
3,3.16,0
4,0.65,0
...,...,...
995,-43.59,1
996,-17.51,1
997,-37.18,1
998,-3.28,1


,Variacion,Fugando combustible
0,44.62,0
1,14.74,0
2,15.51,0
3,-8.27,0
4,42.47,0
...,...,...
995,-121.42,1
996,-37.02,1
997,41.78,1
998,41.88,1


,Variacion,Fugando combustible
0,155.26,0
1,-22.46,0
2,28.24,0
3,-37.57,0
4,-123.92,0
...,...,...
995,-0.64,1
996,6.81,1
997,1.76,1
998,5.69,1


,Variacion,Fugando combustible
0,-19.98,0
1,3.06,0
2,-17.23,0
3,153.68,0
4,36.46,0
...,...,...
995,16.62,1
996,-4.36,1
997,-5.51,1
998,-5.48,1


,Variacion,Fugando combustible
0,38.57,0
1,-3.80,0
2,121.94,0
3,-36.52,0
4,-49.86,0
...,...,...
995,9.58,1
996,-15.22,1
997,-10.10,1
998,6.75,1


...

,Variacion,Fugando combustible
0,13.86,0
1,-0.80,0
2,-2.08,0
3,-82.08,0
4,82.68,0
...,...,...
995,-17.49,1
996,8.55,1
997,-2.91,1
998,5.01,1


,Variacion,Fugando combustible
0,-28.94,0
1,-8.81,0
2,-29.61,0
3,46.18,0
4,-78.55,0
...,...,...
995,5.55,1
996,4.52,1
997,-8.26,1
998,7.98,1


,Variacion,Fugando combustible
0,17.55,0
1,38.07,0
2,62.03,0
3,18.19,0
4,156.28,0
...,...,...
995,-0.19,1
996,10.56,1
997,0.50,1
998,0.78,1


,Variacion,Fugando combustible
0,16.12,0
1,67.61,0
2,-52.52,0
3,-98.03,0
4,20.22,0
...,...,...
995,-3.54,1
996,15.49,1
997,-2.19,1
998,-15.28,1


,Variacion,Fugando combustible
0,-56.69,0
1,142.29,0
2,-10.53,0
3,62.95,0
4,61.07,0
...,...,...
995,13.78,1
996,-8.48,1
997,-5.73,1
998,21.07,1


### Conjunto de datos de prueba

In [11]:
for case in data_dict:
    display(data_dict[case]['test_dataframe'])

,Variacion,Fugando combustible
0,-39.49,0
1,76.95,0
2,-43.75,0
3,58.10,0
4,199.52,0
...,...,...
995,39.00,1
996,50.33,1
997,-75.25,1
998,8.52,1


,Variacion,Fugando combustible
0,-20.04,0
1,-6.75,0
2,18.31,0
3,21.89,0
4,-34.36,0
...,...,...
995,65.18,1
996,-39.54,1
997,29.97,1
998,61.03,1


,Variacion,Fugando combustible
0,12.60,0
1,-94.41,0
2,6.56,0
3,42.94,0
4,36.04,0
...,...,...
995,-13.61,1
996,-4.39,1
997,11.38,1
998,9.09,1


,Variacion,Fugando combustible
0,-33.89,0
1,193.57,0
2,65.60,0
3,16.89,0
4,27.58,0
...,...,...
995,-10.65,1
996,4.61,1
997,-7.05,1
998,14.07,1


,Variacion,Fugando combustible
0,58.02,0
1,121.04,0
2,58.73,0
3,22.45,0
4,32.78,0
...,...,...
995,20.11,1
996,-8.35,1
997,-4.60,1
998,-7.36,1


...

,Variacion,Fugando combustible
0,-61.69,0
1,-10.31,0
2,72.24,0
3,-36.77,0
4,10.39,0
...,...,...
995,1.14,1
996,-3.16,1
997,0.41,1
998,1.01,1


,Variacion,Fugando combustible
0,23.54,0
1,-45.10,0
2,80.80,0
3,33.26,0
4,61.75,0
...,...,...
995,-5.05,1
996,5.32,1
997,8.35,1
998,-9.41,1


,Variacion,Fugando combustible
0,-11.99,0
1,33.92,0
2,40.77,0
3,52.10,0
4,14.66,0
...,...,...
995,-12.63,1
996,5.83,1
997,-3.87,1
998,7.96,1


,Variacion,Fugando combustible
0,-26.51,0
1,-7.50,0
2,2.55,0
3,77.11,0
4,-70.47,0
...,...,...
995,0.42,1
996,3.65,1
997,4.43,1
998,3.78,1


,Variacion,Fugando combustible
0,43.20,0
1,-99.16,0
2,-75.96,0
3,58.12,0
4,-10.42,0
...,...,...
995,-0.39,1
996,1.65,1
997,-11.90,1
998,19.15,1


<!-- TODO -->

## Unificación de conjunto de datos

In [12]:
complete_train_data = pd.concat([element['train_dataframe'] for element in data_dict.values()])

train_data = (
    complete_train_data.drop(columns='Fugando combustible'), 
    complete_train_data['Fugando combustible']
)

In [13]:
segmented_test_data = [
    (element['leak_value'], element['test_dataframe']) for element in data_dict.values()
]

test_data = [(
    'general',
    pd.concat([element['test_dataframe'] for element in data_dict.values()], ignore_index=True)
)]

## Implementación de modelos

### Definición de métricas

In [14]:
def leak_detection_delay(y_true: np.array, y_pred: np.array) -> int:
    real_first_leak_day = 0
    while y_true[real_first_leak_day] != 1: real_first_leak_day += 1

    pred_first_leak_day = real_first_leak_day
    while y_pred[pred_first_leak_day] != 1: pred_first_leak_day += 1

    return (pred_first_leak_day - real_first_leak_day)

In [18]:
def performance_metrics(y_true, y_pred, leak_value = 0.013) -> dict:
    return {
        # Metricas clasicas de ML
            'accuracy_score':   accuracy_score(y_true, y_pred),
            'f1_score':         f1_score(y_true, y_pred),
            'recall_score':     recall_score(y_true, y_pred),
            'precision_score':  precision_score(y_true, y_pred),
            'confusion_matrix': confusion_matrix(y_true, y_pred),
        # Metricas propias del problema
            'false_positives':  confusion_matrix(y_true, y_pred).ravel()[1],
            'leak_delay':       leak_detection_delay(y_true, y_pred),
            'leak_loss':        leak_detection_delay(y_true, y_pred) * leak_value * 60 * 24
    }

### Modelos clásicos

In [15]:
from sklearn.model_selection import GridSearchCV
import pickle as pkl

param_grids_path = '../../pkl/param_grids'

def train_classic_model(classifier_class, param_grid, train_data, cv = 5):
    param_pkl_path = f'{param_grids_path}/{classifier_class.__name__}.pkl'
    classifier = None
    X_train, y_train = train_data
    if os.path.exists(param_pkl_path):  # Param grid already known
        param_grid = pkl.load(open(param_pkl_path, 'rb'))
        classifier = classifier_class(**param_grid)
        classifier.fit(X_train, y_train)
    else:
        grid_search = GridSearchCV(
            estimator = classifier_class(),
            param_grid = param_grid,
            cv = cv
        )
        grid_search.fit(X_train, y_train)
        classifier = grid_search.best_estimator_
        pkl.dump(
            obj = grid_search.best_params_,
            file = open(param_pkl_path, 'wb')
        )
    return classifier

In [16]:
def test_classic_model(classifier, test_list) -> list[dict]:
    results = list()
    for (leak_value, test_data) in test_list:
        X_test = test_data.drop(columns='Fugando combustible')
        y_test = test_data['Fugando combustible']
        y_pred = classifier.predict(X_test)
        results.append(performance_metrics(y_test, y_pred, leak_value))
    return results

In [22]:
def display_performance_table(results: list[dict]) -> None:
    # TODO
    raise NotImplementedError()

##### Random Forest Classifier

In [17]:
from sklearn.ensemble import RandomForestClassifier

rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = train_classic_model(RandomForestClassifier, rf_param_grid, train_data)

In [18]:
display(test_classic_model(rf_model, segmented_test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,0.013000,0.507,0.368758,132,0,0.000000
1,0.054929,0.555,0.443054,117,2,158.194286
2,0.096857,0.629,0.590959,134,0,0.000000
3,0.138786,0.731,0.731269,130,1,199.851429
4,0.180714,0.757,0.767018,138,0,0.000000
5,0.222643,0.790,0.801512,129,0,0.000000
6,0.264571,0.805,0.818267,129,0,0.000000
7,0.306500,0.808,0.823204,134,0,0.000000
8,0.348429,0.815,0.832882,141,0,0.000000
9,0.390357,0.813,0.827331,130,0,0.000000


In [19]:
display(test_classic_model(rf_model, test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,general,0.758933,0.764614,1914,0,0


##### Naive Bayes Classifier

In [20]:
from sklearn.naive_bayes import GaussianNB

gnb_param_grid = {
    'var_smoothing': [1e-11, 1e-10, 1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

gnb_model = train_classic_model(GaussianNB, gnb_param_grid, train_data)

In [21]:
display(test_classic_model(gnb_model, segmented_test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,0.013000,0.507,0.288600,88,1,18.720000
1,0.054929,0.596,0.476684,83,1,79.097143
2,0.096857,0.616,0.517588,85,0,0.000000
3,0.138786,0.585,0.441454,74,1,199.851429
4,0.180714,0.562,0.398352,78,0,0.000000
5,0.222643,0.537,0.359613,88,0,0.000000
6,0.264571,0.536,0.335244,76,0,0.000000
7,0.306500,0.508,0.295129,90,0,0.000000
8,0.348429,0.511,0.281938,80,0,0.000000
9,0.390357,0.494,0.258065,89,0,0.000000


In [22]:
display(test_classic_model(gnb_model, test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,general,0.5252,0.322489,1242,1,0


##### KNN Classifier

In [23]:
from sklearn.neighbors import KNeighborsClassifier

knn_param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

knn_model = train_classic_model(KNeighborsClassifier, knn_param_grid, train_data)

In [24]:
display(test_classic_model(knn_model, segmented_test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,0.013000,0.507,0.378310,138,0,0.000000
1,0.054929,0.559,0.458896,123,2,158.194286
2,0.096857,0.640,0.612903,140,0,0.000000
3,0.138786,0.732,0.732000,129,1,199.851429
4,0.180714,0.724,0.733591,151,0,0.000000
5,0.222643,0.785,0.795043,127,0,0.000000
6,0.264571,0.798,0.809793,127,0,0.000000
7,0.306500,0.791,0.807018,141,0,0.000000
8,0.348429,0.788,0.807623,152,0,0.000000
9,0.390357,0.796,0.810056,134,0,0.000000


In [25]:
display(test_classic_model(knn_model, test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,general,0.747067,0.752447,1985,0,0


##### XGBoost Classifier

In [26]:
from xgboost import XGBClassifier

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

xgb_model = train_classic_model(XGBClassifier, xgb_param_grid, train_data)

In [27]:
display(test_classic_model(xgb_model, segmented_test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,0.013000,0.510,0.358639,122,0,0.000000
1,0.054929,0.547,0.425856,116,4,316.388571
2,0.096857,0.624,0.569794,120,0,0.000000
3,0.138786,0.722,0.722555,135,1,199.851429
4,0.180714,0.765,0.775120,135,0,0.000000
5,0.222643,0.806,0.816981,122,0,0.000000
6,0.264571,0.803,0.816744,131,0,0.000000
7,0.306500,0.827,0.840259,123,0,0.000000
8,0.348429,0.828,0.843066,129,0,0.000000
9,0.390357,0.836,0.848708,119,0,0.000000


In [28]:
display(test_classic_model(xgb_model, test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,general,0.766933,0.771414,1820,0,0


##### _Ensemble Classifier_

In [29]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier()),
        ('nb', GaussianNB()),
        ('knn', KNeighborsClassifier()),
        ('XGB', XGBClassifier())
    ],
    voting='hard'
)

voting_clf.fit(train_data[0], train_data[1])

None

In [30]:
display(test_classic_model(voting_clf, segmented_test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,0.013000,0.501,0.309820,106,0,0.000000
1,0.054929,0.540,0.388298,101,2,158.194286
2,0.096857,0.625,0.551971,101,0,0.000000
3,0.138786,0.709,0.684039,101,1,199.851429
4,0.180714,0.721,0.710280,116,1,260.228571
5,0.222643,0.766,0.757261,94,0,0.000000
6,0.264571,0.758,0.755061,110,0,0.000000
7,0.306500,0.762,0.757143,104,0,0.000000
8,0.348429,0.770,0.771372,113,0,0.000000
9,0.390357,0.766,0.764588,109,0,0.000000


In [31]:
display(test_classic_model(voting_clf, test_data))

,leak value,accuracy,f1 score,false positives,leak detection delay (days),leak loss (liters)
0,general,0.7182,0.69861,1551,0,0


### Modelos _Deep Learning_

In [32]:
def eval_dl_model(model, test_list):
    result_dict = {}
    for leak_value, test_dataframe in test_list:
        X_test = test_dataframe.drop(columns='Fugando combustible')
        y_test = test_dataframe['Fugando combustible']
        
        y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

        accuracy = accuracy_score(y_test, y_pred)
        leak_delay = leak_detection_delay(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
        leak_loss = (leak_value * 60 * 24 * leak_delay) if type(leak_value) == float else 0

        result_dict[leak_value] = (accuracy, f1, fp, leak_delay, leak_loss)

    return pd.DataFrame.from_dict(
        data = result_dict,
        orient = 'index',
        columns = ['accuracy_score', 'f1_score', 'false positives', 'leak_detection_delay', 'leak_loss']
    )

#### Único modelo de _Deep Learning_

In [33]:
X_train, y_train = train_data

##### Modelo unicapa

In [34]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(15, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

display(eval_dl_model(model, segmented_test_data))

,accuracy_score,f1_score,false positives,leak_detection_delay,leak_loss
0.013000,0.508,0.385000,141,0,0.000000
0.054929,0.529,0.406053,127,4,316.388571
0.096857,0.603,0.548350,133,1,139.474286
0.138786,0.702,0.702595,145,1,199.851429
0.180714,0.750,0.761450,144,0,0.000000
0.222643,0.787,0.801120,137,0,0.000000
0.264571,0.800,0.817185,142,0,0.000000
0.306500,0.820,0.837251,138,0,0.000000
0.348429,0.830,0.848754,142,0,0.000000
0.390357,0.818,0.836625,143,0,0.000000


##### Modelo multicapa

In [35]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
    tf.keras.layers.LSTM(25, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(40, return_sequences=True, dropout=0.2),
    tf.keras.layers.LSTM(60, dropout=0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x = X_train,
    y = y_train,
    epochs = 50,
    batch_size = 8192,
    verbose = 0
)

display(eval_dl_model(model, segmented_test_data))

,accuracy_score,f1_score,false positives,leak_detection_delay,leak_loss
0.013000,0.525,0.429772,149,0,0.000000
0.054929,0.549,0.470035,146,4,316.388571
0.096857,0.620,0.594017,153,0,0.000000
0.138786,0.712,0.724138,161,1,199.851429
0.180714,0.758,0.778388,162,0,0.000000
0.222643,0.791,0.810517,151,0,0.000000
0.264571,0.792,0.815275,162,0,0.000000
0.306500,0.814,0.836268,156,0,0.000000
0.348429,0.827,0.848644,153,0,0.000000
0.390357,0.820,0.842105,155,0,0.000000


#### Múltiples modelos de _Deep Learning_

In [36]:
results = {}
for case in data_dict:
    X_train = data_dict[case]['train_dataframe'].drop(columns='Fugando combustible')
    y_train = data_dict[case]['train_dataframe']['Fugando combustible']
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1], 1)),
        tf.keras.layers.LSTM(15, dropout=0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    history = model.fit(
        x = X_train,
        y = y_train,
        epochs = 50,
        batch_size = 8192,
        verbose = 0
    )

    leak_value = data_dict[case]['leak_value']

    X_test = data_dict[case]['test_dataframe'].drop(columns='Fugando combustible')
    y_test = data_dict[case]['test_dataframe']['Fugando combustible']
    
    y_pred = (model.predict(X_test, verbose=0) > 0.5).astype(int)

    accuracy = accuracy_score(y_test, y_pred)
    leak_delay = leak_detection_delay(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    leak_loss = (leak_value * 60 * 24 * leak_delay) if type(leak_value) == float else 0

    results[leak_value] = (accuracy, f1, fp, leak_delay, leak_loss)
    # model.save(f'../../pkl/models/{case}.keras')

display(pd.DataFrame.from_dict(
    data = results,
    orient = 'index',
    columns = ['accuracy_score', 'f1_score', 'false positives', 'leak_detection_delay', 'leak_loss']
))

,accuracy_score,f1_score,false positives,leak_detection_delay,leak_loss
0.013000,0.497,0.564502,324,1,18.720000
0.054929,0.638,0.663569,214,1,79.097143
0.096857,0.657,0.702515,243,0,0.000000
0.138786,0.719,0.769104,244,0,0.000000
0.180714,0.586,0.627698,258,0,0.000000
0.222643,0.724,0.771144,236,0,0.000000
0.264571,0.677,0.710833,215,0,0.000000
0.306500,0.554,0.640323,338,0,0.000000
0.348429,0.628,0.712519,328,0,0.000000
0.390357,0.705,0.755187,245,0,0.000000
